In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.constraint_graph import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *

gtimer = GlobalTimer.instance()
gtimer.reset()

# initialize numbers

In [2]:
from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT

In [3]:
from pkg.data_collecting.sampling import sample_Trbt, show_workspace

In [4]:
crob = CombinedRobot(connection_list=(False, False))

connection_list
(False, False)


In [5]:
Nrbt = len(crob.robot_names) # 2
Njoints = len(crob.joint_names) # 13
Ws, Ds, Hs = WDH = (3,3,3)
WDH_MIN_RBT, WDH_MAX_RBT = (0.75,0.75,0), (2.25,2.25,0.5)
CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
MIN_DIST_ROBOT = 1
L_CELL = 0.2
Nw, Nd, Nh = Nwdh = int(Ws/L_CELL), int(Ds/L_CELL), int(Hs/L_CELL)
print("scene size: {} ({},{},{})".format(Nw*Nd*Nh, Nw, Nd, Nh))

scene size: 3375 (15,15,15)


In [6]:
Trbt= sample_Trbt(Nrbt, crob.robot_names, WDH_MIN_RBT, WDH_MAX_RBT, MIN_DIST_ROBOT)

# Build Scene

## put robot

In [7]:
cam = None

# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, Trbt, crob.joint_names)
time.sleep(2)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


In [8]:
# set graph
if "graph" in locals():
    graph.clear_markers()
    graph.clear_highlight()
    graph.ghnd.clear()
    graph.__init__(urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                   urdf_content=urdf_content, combined_robot=crob)
    graph.set_cam_robot_collision(_add_cam_poles=False)
    graph.set_rviz()
else:
    graph = ConstraintGraph(urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                            urdf_content=urdf_content, combined_robot=crob)
    graph.set_camera(cam)
    graph.set_cam_robot_collision(_add_cam_poles=False)
    graph.set_rviz()
    
    # start UI
    ui_broker = UIBroker(graph)
    ui_broker.start_server()
    
    # set rviz
    graph.set_rviz(crob.home_pose)
    ui_broker.set_tables()

Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## show workspace

In [9]:
# show_workspace(graph, Nwdh, CENTER, L_CELL)

## init joints

In [10]:
joint_lims = []
for jname in crob.joint_names:
    joint = graph.urdf_content.joint_map[jname]
    joint_lims.append([joint.limit.upper, joint.limit.lower])
joint_lims=np.array(joint_lims)
Qmax, Qmin = joint_lims[:,0],joint_lims[:,1]
Q_s = np.random.uniform(Qmin, Qmax)
graph.show_pose(Q_s)
time.sleep(1)
graph.show_pose(Q_s)
time.sleep(1)
graph.show_pose(Q_s)

In [11]:
from pkg.data_collecting.sampling import *

In [12]:
coll_mat = get_collision_mat(Nwdh, L_CELL, crob, graph.ghnd, Q_s)

In [13]:
import os
import sys

sys.path.append(os.path.join(os.environ["TAMP_ETASL_DIR"], "openGJK/lib"))
import openGJKlib as oGJK

In [14]:
from pkg.geometry.geometry import DEFAULT_VERT_DICT, GEOTYPE
from pkg.utils.rotation_utils import Rot_rpy
import numpy as np

In [ ]:
from ics import 

In [16]:
coll_boxes = []
for iw in range(Nw):
    for id in range(Nd):
        for ih in range(Nh):
            if any(coll_mat[iw,id,ih]<0):
                coll_boxes.append([iw,id,ih])

In [17]:
for cbox in coll_boxes:
    graph.add_geometry(graph.ghnd.create_safe(
        name="col_cell_{}_{}_{}".format(*cbox), link_name="base_link", gtype=GEOTYPE.BOX,
        center=tuple(np.multiply(cbox, L_CELL)+L_CELL/2), rpy=(0, 0, 0), dims=(L_CELL, L_CELL, L_CELL),
        color=(0.9, 0, 0.2, 0.2), display=True, collision=False, fixed=True))